<a href="https://colab.research.google.com/github/donghwi2022/ds-sa-cp2-phase2/blob/%EC%8B%A0%EB%8F%99%ED%9C%98/model_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 import 및 사용 데이터 load

In [1]:
!pip install --user annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
import annoy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_parquet("/content/drive/MyDrive/cp2_phase2_data/fashion_logs.parquet")
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1201009 entries, 0 to 1254584
Data columns (total 23 columns):
 #   Column              Non-Null Count    Dtype   
---  ------              --------------    -----   
 0   created_at          1201009 non-null  category
 1   session_id          1201009 non-null  int32   
 2   payment_method      1201009 non-null  category
 3   promo_amount        1201009 non-null  int32   
 4   promo_code          1201009 non-null  category
 5   shipment_fee        1201009 non-null  int32   
 6   total_amount        1201009 non-null  int32   
 7   product_id          1201009 non-null  int32   
 8   productDisplayName  1201009 non-null  category
 9   quantity            1201009 non-null  int32   
 10  item_price          1201009 non-null  int32   
 11  product_gender      1201009 non-null  category
 12  baseColor           1201009 non-null  category
 13  season              1201009 non-null  category
 14  year                1201009 non-null  int32   
 15

In [5]:
data.head()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
0,2018-07-29T15:22:01,194913,Debit Card,1415,WEEKENDSERU,10000,199832,54728,Vans Men Black Shoes,1,...,Summer,2012,Casual,Footwear Shoes Casual Shoes,5868,F,2000-08-20,Android,Jakarta Raya,2018-07-28
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,...,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,...,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28


# 2. CB 모델 구축

## 1. 학습 데이터와 테스트 데이터 분리
- 학습과 테스트를 진행하기 위해서는 최소 2개의 데이터가 필요하므로, 데이터가 1개뿐인 고객은 제거
- ~~시간 순서로 데이터를 분할하기 위해 라이브러리 미사용~~ <br>
    → 시간이 너무 오래걸려 라이브러리 사용



In [6]:
data.groupby('product_id').count()

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,productDisplayName,quantity,item_price,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
product_id,,,,,,,,,,,,,,,,,,,,,
1163,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
1164,21,21,21,21,21,21,21,21,21,21,...,21,21,21,21,21,21,21,21,21,21
1165,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29
1525,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31
1526,24,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18
59996,27,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
59998,26,26,26,26,26,26,26,26,26,26,...,26,26,26,26,26,26,26,26,26,26


In [7]:
# 데이터가 1개뿐인 고객 제거
data_by_customer = data.groupby('customer_id').count()
only1_data = data_by_customer[data_by_customer['created_at'] < 2].index
data_new = data[~data['customer_id'].isin(only1_data)]
data_new

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
1,2018-07-30T12:40:22,155874,Credit Card,0,No,10000,155526,16193,Puma Men Knitted Vest Green Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
2,2018-09-15T11:51:17,489413,OVO,0,No,10000,550696,53686,Kiara Women Purple & Yellow Handbag,4,...,Summer,2012,Casual,Accessories Bags Handbags,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
3,2018-11-01T11:23:48,627839,Credit Card,0,No,0,271012,20228,Wrangler Women Cable Red Sweater,1,...,Fall,2011,Casual,Apparel Topwear Sweaters,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
4,2018-12-18T11:20:30,742846,Credit Card,0,No,0,198753,55220,Lakme Absolute Forever Silk Chestnut Lip Liner 03,1,...,Spring,2017,Casual,Personal Care Lips Lip Liner,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
5,2019-02-03T11:25:55,424328,Debit Card,6369,WEEKENDMANTAP,5000,181865,59620,FNF Green & Black Wedding Collection Sari,1,...,Summer,2012,Ethnic,Apparel Saree Sarees,4774,F,1996-03-15,Android,Sulawesi Selatan,2018-07-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254573,2022-05-09T19:43:52,276163,Credit Card,5177,BUYMORE,0,387326,4788,ADIDAS Men's Demon White T-shirt,1,...,Summer,2011,Sports,Apparel Topwear Tshirts,99675,F,1991-08-17,iOS,Yogyakarta,2022-05-06
1254574,2022-06-08T02:44:08,470126,Credit Card,0,No,0,237077,15331,Nike Men Solid White Shorts,1,...,Summer,2012,Sports,Apparel Bottomwear Shorts,99675,F,1991-08-17,iOS,Yogyakarta,2022-05-06
1254575,2022-07-07T09:48:29,35755,Credit Card,3388,XX2022,0,485334,35318,Fabindia Women Anusuya Silver Earrings,1,...,Winter,2016,Ethnic,Accessories Jewellery Earrings,99675,F,1991-08-17,iOS,Yogyakarta,2022-05-06
1254576,2016-09-22T00:46:56,33802,Credit Card,0,No,5000,531721,20764,Carlton London Women Red Casual Heels,1,...,Winter,2012,Casual,Footwear Shoes Heels,16528,M,1983-07-27,Android,Kepulauan Riau,2016-09-20


In [8]:
data_new['customer_id'].value_counts()

43202    807
29496    749
82237    716
10167    680
69740    673
        ... 
65066      2
14506      2
18847      2
34650      2
16528      2
Name: customer_id, Length: 41766, dtype: int64

In [9]:
train_data, test_data = train_test_split(data_new, test_size=0.2, random_state=2)

In [10]:
train_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
618302,2022-02-25T03:41:38,768076,OVO,0,No,10000,144170,43859,Royal Diadem Golden Earrings,1,...,Summer,2012,Casual,Accessories Jewellery Earrings,89369,M,1990-04-26,Android,Kepulauan Riau,2020-09-06
475896,2019-05-01T00:00:39,67118,Gopay,0,No,10000,816770,14308,ADIDAS Men Solid Black Jackets,1,...,Fall,2011,Casual,Apparel Topwear Jackets,43027,F,1988-12-09,Android,Sumatera Barat,2017-04-15
258,2019-09-07T23:41:55,432651,LinkAja,8912,WEEKENDMANTAP,5000,1701096,32274,Vishudh Women Printed Brown Kurta,1,...,Summer,2012,Ethnic,Apparel Topwear Kurtas,76966,F,1985-07-21,Android,Kalimantan Tengah,2018-07-28
83620,2020-05-19T15:30:32,298757,Credit Card,3368,XX2022,10000,241344,12385,Reid & Taylor Men Solid Pink Shirts,1,...,Fall,2011,Casual,Apparel Topwear Shirts,13242,F,2002-12-13,Android,Kalimantan Timur,2019-08-03
474820,2021-12-16T02:11:09,192030,Gopay,2917,AZ2022,10000,143700,31440,HIghlander Men Grey Trousers,1,...,Summer,2012,Casual,Apparel Bottomwear Trousers,69038,M,1998-01-18,Android,Jawa Tengah,2019-01-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159075,2020-03-19T19:57:57,336710,LinkAja,0,No,0,736496,5685,Skechers Men's Traffics Black Shoe,1,...,Winter,2015,Casual,Footwear Shoes Casual Shoes,34268,M,1997-07-06,Android,Jawa Tengah,2018-02-28
1011510,2021-10-29T19:58:32,695089,Credit Card,3017,AZ2022,10000,1349328,12934,Classic Polo Men Rugby Scrum Blue Tshirts,5,...,Fall,2011,Casual,Apparel Topwear Tshirts,82817,F,2001-10-31,iOS,Yogyakarta,2017-06-15
1191184,2021-11-05T21:33:43,672070,OVO,4830,XX2022,15000,661535,32577,ONLY Women Toronto Grey Printed Dress,1,...,Summer,2012,Casual,Apparel Dress Dresses,76759,F,1996-07-10,iOS,Sulawesi Selatan,2017-02-17
212949,2019-11-21T14:35:25,179622,OVO,3886,AZ2022,10000,143112,45654,Park Avenue Men Navy Blue Brief,1,...,Summer,2016,Casual,Apparel Innerwear Briefs,5459,F,1999-07-20,iOS,Jakarta Raya,2018-07-29


In [11]:
test_data

,created_at,session_id,payment_method,promo_amount,promo_code,shipment_fee,total_amount,product_id,productDisplayName,quantity,...,season,year,usage,Category,customer_id,customer_gender,birthdate,device_type,home_location,first_join_date
929183,2020-07-09T21:00:07,167980,Debit Card,0,No,15000,175527,57318,Amante Black Full-Coverage Lace Bra BCFR02,1,...,Winter,2015,Casual,Apparel Innerwear Bra,61146,F,2005-07-31,Android,Lampung,2019-06-29
545389,2021-08-07T21:21:12,609198,Debit Card,1149,WEEKENDMANTAP,10000,1916691,12076,W Women Printed Beige Kurtas,1,...,Fall,2011,Ethnic,Apparel Topwear Kurtas,88868,F,1994-12-10,Android,Kalimantan Tengah,2020-08-22
115760,2019-12-10T18:32:16,378910,OVO,0,No,0,1196243,40127,Gini and Jony Girls Roseatte Black Dress,1,...,Summer,2012,Casual,Apparel Dress Dresses,30849,M,1998-01-01,iOS,Jawa Barat,2019-07-20
917235,2019-08-07T20:38:33,460356,LinkAja,5757,XX2022,10000,142636,7162,Scullers Men's Scul Purple T-shirt,1,...,Fall,2011,Casual,Apparel Topwear Tshirts,27524,M,1992-11-11,Android,Jawa Tengah,2016-12-18
778100,2022-04-22T08:54:13,316326,Gopay,0,No,10000,299729,29935,Police Men White Digital Watch PL12898JS-02A,1,...,Winter,2016,Casual,Accessories Watches Watches,48963,M,1989-08-31,Android,Kepulauan Riau,2021-03-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807625,2020-04-09T06:00:22,754376,OVO,12136,STARTUP,10000,276466,27075,Catwalk Women Black Heels,1,...,Winter,2015,Casual,Footwear Shoes Heels,30403,F,1997-09-09,Android,Jakarta Raya,2019-08-10
915131,2021-08-28T09:18:24,479022,OVO,0,No,10000,4781233,15224,ADIDAS Originals Men Vespa PK LO Black Casual ...,2,...,Fall,2011,Casual,Footwear Shoes Casual Shoes,63914,F,2003-05-29,iOS,Jawa Timur,2021-08-28
170797,2022-02-02T07:15:49,576430,Gopay,0,No,15000,245568,44663,Wills Lifestyle Women Brown Trousers,1,...,Summer,2012,Formal,Apparel Bottomwear Trousers,10593,F,1982-07-24,Android,Nusa Tenggara Barat,2016-10-30
923581,2022-05-05T19:58:42,657910,Credit Card,3864,AZ2022,10000,7667225,24702,Puma Men Mid Spill Green Casual Shoes,2,...,Summer,2011,Casual,Footwear Shoes Casual Shoes,32981,M,1988-02-29,Android,Jakarta Raya,2022-04-30


## 2. 코사인유사도 계산

In [24]:
# 사용할 feature 선별
product_data = data_new[['product_id', 'product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].drop_duplicates('product_id')
product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']] = product_data[['product_gender', 'baseColor', 'season', 'year', 'usage', 'Category']].astype('str')
product_data.reset_index(inplace=True)
product_data

,index,product_id,product_gender,baseColor,season,year,usage,Category
0,1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters
1,2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags
2,3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters
3,4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner
4,5,59620,Women,Green,Summer,2012,Ethnic,Apparel Saree Sarees
...,...,...,...,...,...,...,...,...
44440,437886,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas
44441,440721,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes
44442,457830,34019,Boys,Black,Summer,2012,Casual,Apparel Topwear Tshirts
44443,462975,36651,Women,Red,Summer,2012,Casual,Accessories Wallets Wallets


In [25]:
product_data['features'] = product_data['product_gender']+' ' +product_data['baseColor']+' ' +product_data['season']+' ' +product_data['year']+' ' +product_data['usage']+' ' +product_data['Category']
product_data

,index,product_id,product_gender,baseColor,season,year,usage,Category,features
0,1,16193,Men,Green,Fall,2011,Casual,Apparel Topwear Sweaters,Men Green Fall 2011 Casual Apparel Topwear Swe...
1,2,53686,Women,Purple,Summer,2012,Casual,Accessories Bags Handbags,Women Purple Summer 2012 Casual Accessories Ba...
2,3,20228,Women,Red,Fall,2011,Casual,Apparel Topwear Sweaters,Women Red Fall 2011 Casual Apparel Topwear Swe...
3,4,55220,Women,Brown,Spring,2017,Casual,Personal Care Lips Lip Liner,Women Brown Spring 2017 Casual Personal Care L...
4,5,59620,Women,Green,Summer,2012,Ethnic,Apparel Saree Sarees,Women Green Summer 2012 Ethnic Apparel Saree S...
...,...,...,...,...,...,...,...,...,...
44440,437886,59916,Women,Brown,Summer,2012,Ethnic,Apparel Topwear Kurtas,Women Brown Summer 2012 Ethnic Apparel Topwear...
44441,440721,2503,Men,Brown,Fall,2012,Casual,Footwear Shoes Casual Shoes,Men Brown Fall 2012 Casual Footwear Shoes Casu...
44442,457830,34019,Boys,Black,Summer,2012,Casual,Apparel Topwear Tshirts,Boys Black Summer 2012 Casual Apparel Topwear ...
44443,462975,36651,Women,Red,Summer,2012,Casual,Accessories Wallets Wallets,Women Red Summer 2012 Casual Accessories Walle...


In [38]:
# tf-idf 벡터화
tfidf = TfidfVectorizer()
product_tfidf = tfidf.fit_transform(product_data['features'])
product_tfidf = pd.DataFrame(product_tfidf.todense(), columns = tfidf.get_feature_names())
product_tfidf

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,...,waistcoat,wallets,wash,watches,water,white,winter,women,wristbands,yellow
0,0.0,0.0,0.0,0.0,0.251266,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.243786,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.227443,0.0,0.0
2,0.0,0.0,0.0,0.0,0.252014,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.216338,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.130091,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.194092,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.181081,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44440,0.0,0.0,0.0,0.0,0.000000,0.256879,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.239658,0.0,0.0
44441,0.0,0.0,0.0,0.0,0.000000,0.240922,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
44442,0.0,0.0,0.0,0.0,0.000000,0.270176,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
44443,0.0,0.0,0.0,0.0,0.000000,0.177587,0.0,0.0,0.0,0.0,...,0.0,0.861378,0.0,0.0,0.0,0.0,0.0,0.165682,0.0,0.0


In [40]:
annoy_index = annoy.AnnoyIndex(f=product_tfidf.shape[1], metric='angular')
product_data_arr = product_tfidf.to_numpy()

for idx in range(product_data_arr.shape[0]):
    vector = product_data_arr[idx, :]
    annoy_index.add_item(idx, vector)

In [41]:
annoy_index.build(n_trees=5) 
annoy_index.save('test.ann')

True

In [42]:
product_tfidf.iloc[0, :]

2007          0.000000
2008          0.000000
2009          0.000000
2010          0.000000
2011          0.251266
                ...   
white         0.000000
winter        0.000000
women         0.000000
wristbands    0.000000
yellow        0.000000
Name: 0, Length: 267, dtype: float64

In [43]:
med_value = product_tfidf.iloc[0, :]

In [46]:
get_nns_list = annoy_index.get_nns_by_vector(vector=med_value, n=6, include_distances=False)
get_nns_list

[0, 2427, 20899, 34290, 37859, 38268]

In [15]:
data.shape[1]

23

# 3. 성능 평가